# AIMO3 Elle Submission - 11 Dec 2025
## Single-Pass Strategy with Elle-72B-Ultimate

**Architecture:** Single model, maximum KV cache utilization, adaptive sampling

**Kaggle Inputs:**
- `/kaggle/input/elle-72b-ultimate` - Elle-72B-Ultimate model
- `/kaggle/input/aimo3-vllm-wheels` - ALL offline wheels (vLLM + deps)
- `/kaggle/input/ai-mathematical-olympiad-progress-prize-3/` - Competition data

**Strategy:**
- Problems served ONCE in random order → maximize each attempt
- Prefix caching shares system prompts across all generations
- Adaptive batch size: more samples early, fewer as time depletes
- Temperature annealing: HOT→WARM→COOL
- Early consensus termination: stop when 3+ agree

In [ ]:
# Cell 1: Environment Cleanup
%pip uninstall --yes tensorflow matplotlib keras scikit-learn 2>/dev/null || true

In [ ]:
# Cell 2: Core Imports
import os, sys, gc, time, random, warnings, re, statistics, tempfile, subprocess, math
from collections import Counter, defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Optional, List, Dict, Any, Tuple
from dataclasses import dataclass
from enum import Enum

sys.setrecursionlimit(20000)
warnings.simplefilter('ignore')
os.environ.update({
    "TRANSFORMERS_NO_TF": "1",
    "TRANSFORMERS_NO_FLAX": "1",
    "CUDA_VISIBLE_DEVICES": "0",
    "TOKENIZERS_PARALLELISM": "false",
    "PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION": "python"
})

SEED = 42
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import torch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
assert torch.cuda.is_available(), "GPU NOT AVAILABLE"
print(f"GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")

import polars as pl

# Timing
START_TIME = time.time()
TOTAL_BUDGET = (4*60+40)*60  # 4h40m
CUTOFF_TIME = START_TIME + TOTAL_BUDGET
PANIC_TIME = 300

# Answer bounds
ANSWER_MIN = 0
ANSWER_MAX = 999999
FALLBACK = 0

print(f"Budget: {TOTAL_BUDGET//60}min")

In [ ]:
# Cell 3: Install offline wheels & imports
import glob

# SINGLE wheel directory with everything
WHEEL_DIR = "/kaggle/input/aimo3-vllm-wheels"
COMP_PATH = "/kaggle/input/ai-mathematical-olympiad-progress-prize-3"

def find_wheels(base_path):
    """Find all .whl files recursively."""
    if not os.path.exists(base_path):
        return []
    wheels = glob.glob(f"{base_path}/**/*.whl", recursive=True)
    if not wheels:
        wheels = glob.glob(f"{base_path}/*.whl")
    return wheels

# Debug: show wheel directory contents
print(f"\n{WHEEL_DIR}:")
if os.path.exists(WHEEL_DIR):
    wheels = find_wheels(WHEEL_DIR)
    print(f"  Found {len(wheels)} wheels")
    for w in wheels[:15]:
        print(f"  📄 {os.path.basename(w)}")
    if len(wheels) > 15:
        print(f"  ... and {len(wheels) - 15} more")
else:
    print("  ❌ NOT FOUND")

# Install from offline wheels (--no-index = fully offline, --no-deps = don't try to resolve)
print("\n--- Installing wheels ---")
wheels = find_wheels(WHEEL_DIR)
if wheels:
    wheel_dir = os.path.dirname(wheels[0]) if wheels else WHEEL_DIR
    
    # Install vLLM first (no deps since we have them all)
    print("Installing vLLM...")
    result = subprocess.run([
        sys.executable, "-m", "pip", "install", "-q",
        "--no-index", "--no-deps", f"--find-links={wheel_dir}", "vllm"
    ], capture_output=True, text=True)
    if result.returncode != 0:
        print(f"  vLLM: {result.stderr[:200]}")
    
    # Install all other wheels - GGUF FIRST (vLLM needs it)
    print("Installing dependencies...")
    all_packages = [
        "gguf", "prometheus-fastapi-instrumentator", "prometheus-client", "fastapi", "starlette",
        "uvicorn", "pydantic", "pydantic-core", "anyio", "httpx", "httpcore", "aiohttp",
        "msgspec", "pyzmq", "ray", "xformers", "compressed-tensors", "nvidia-ml-py",
        "tiktoken", "einops", "accelerate", "peft", "transformers", "tokenizers",
        "safetensors", "huggingface-hub", "lm-format-enforcer", "partial-json-parser",
        "sentencepiece", "protobuf", "pillow", "pyyaml", "filelock", "tqdm", "requests",
        "certifi", "psutil", "py-cpuinfo", "packaging", "typing-extensions",
        "annotated-types", "click", "h11", "idna", "sniffio", "distro", "jiter"
    ]
    
    result = subprocess.run([
        sys.executable, "-m", "pip", "install", "-q",
        "--no-index", "--no-deps", f"--find-links={wheel_dir}"
    ] + all_packages, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"  Some deps failed (may be OK): {result.stderr[:300]}")
    
    print("✓ Wheels installed")
else:
    print("⚠️ No wheels found - trying online install")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "vllm==0.6.2"], 
                   capture_output=True)

# Add competition path
if os.path.exists(COMP_PATH) and COMP_PATH not in sys.path:
    sys.path.insert(0, COMP_PATH)
    print(f"✓ Added {COMP_PATH} to path")

from vllm import LLM, SamplingParams
from transformers import set_seed
import kaggle_evaluation.aimo_3_inference_server
set_seed(SEED)
print("\n✓ All imports ready")

In [ ]:
# Cell 4: Model Path Detection - ELLE ONLY
# Single model strategy - no fallbacks, maximize Elle's capabilities
MODEL_PATHS = [
    "/kaggle/input/elle-72b-ultimate/transformers/transformers/1",  # Primary
    "/kaggle/input/elle-72b-ultimate/transformers/1",               # Alt structure
    "/kaggle/input/elle-72b-ultimate/1",                            # Version subfolder
    "/kaggle/input/elle-72b-ultimate",                              # Root
]

def find_model():
    for p in MODEL_PATHS:
        if os.path.exists(p):
            files = os.listdir(p) if os.path.isdir(p) else []
            # Look for model files
            has_model = any('safetensors' in f or 'config.json' in f or 'model' in f.lower() for f in files)
            if has_model:
                print(f"✓ Found Elle: {p}")
                print(f"  Files: {files[:8]}...")
                return p
            # Check subdirs
            for sub in files:
                subp = os.path.join(p, sub)
                if os.path.isdir(subp):
                    subfiles = os.listdir(subp)
                    if any('safetensors' in f or 'config.json' in f for f in subfiles):
                        print(f"✓ Found Elle: {subp}")
                        print(f"  Files: {subfiles[:8]}...")
                        return subp
    
    # Debug: show what's actually there
    base = "/kaggle/input/elle-72b-ultimate"
    if os.path.exists(base):
        print(f"DEBUG - Contents of {base}:")
        for item in os.listdir(base):
            full = os.path.join(base, item)
            if os.path.isdir(full):
                print(f"  📁 {item}/ -> {os.listdir(full)[:5]}")
            else:
                print(f"  📄 {item}")
    
    raise RuntimeError("Elle model not found! Check Kaggle inputs.")

LLM_MODEL_PATH = find_model()

# Detect sample vs full test
IS_SAMPLE = True
TEST_PATH = f"{COMP_PATH}/test.csv"
if os.path.exists(TEST_PATH):
    test_df = pl.read_csv(TEST_PATH)
    IS_SAMPLE = len(test_df) <= 5
    print(f"Test set: {len(test_df)} problems ({'sample' if IS_SAMPLE else 'FULL'})")

print(f"Model: {LLM_MODEL_PATH}")

In [ ]:
# Cell 5: Load vLLM - MAXIMUM KV CACHE UTILIZATION
# Single-pass strategy: maximize memory for KV cache, enable prefix caching

MAX_NUM_SEQS = 48  # Higher parallelism for batch generation
MAX_MODEL_LEN = 16384  # Full context for complex problems

# Aggressive KV cache config
model_kwargs = {
    "max_num_seqs": MAX_NUM_SEQS,
    "max_model_len": MAX_MODEL_LEN,
    "trust_remote_code": True,
    "tensor_parallel_size": 1,
    "gpu_memory_utilization": 0.97,  # Max VRAM for KV cache
    "seed": SEED,
    "enable_prefix_caching": True,   # CRITICAL: shares system prompts
    "swap_space": 4,                 # GB swap for overflow
    "disable_log_stats": True,       # Reduce overhead
}

print("Loading Elle-72B-Ultimate...")
print(f"  KV cache dtype: FP8 (2x capacity)")
print(f"  Prefix caching: ENABLED")
print(f"  GPU memory: 97%")

try:
    llm = LLM(
        LLM_MODEL_PATH, 
        kv_cache_dtype="fp8",        # FP8 = 2x KV cache capacity
        enforce_eager=False,          # Enable CUDA graphs
        **model_kwargs
    )
    print("✓ Model loaded with FP8 KV cache")
except Exception as e:
    print(f"FP8 failed ({e}), trying FP16...")
    model_kwargs["gpu_memory_utilization"] = 0.95
    llm = LLM(LLM_MODEL_PATH, **model_kwargs)
    print("✓ Model loaded with FP16 KV cache")

tokenizer = llm.get_tokenizer()

# Pre-warm KV cache with system prompts
SYSTEM_CACHE_PROMPTS = [
    "You are an expert mathematical problem solver.",
    "You are a symbolic computation expert using sympy.",
    "You are a computational enumerator.",
    "You are a mathematical olympiad expert.",
]
print(f"Tokenizer ready | Vocab: {tokenizer.vocab_size}")

In [ ]:
# Cell 6: Problem Classifier
class ProblemType(Enum):
    NUMBER_THEORY = "nt"
    COMBINATORICS = "comb"
    GEOMETRY = "geom"
    ALGEBRA = "alg"
    MIXED = "mixed"

@dataclass
class ProblemProfile:
    primary_type: ProblemType
    modulo_target: Optional[int]
    confidence: float

def classify_problem(question: str) -> ProblemProfile:
    q = question.lower()
    scores = {t: 0.0 for t in ProblemType}
    
    for kw in ['remainder', 'modulo', 'mod ', 'divisible', 'prime', 'gcd', 'lcm', 'factor', 'digit']:
        if kw in q: scores[ProblemType.NUMBER_THEORY] += 2
    for kw in ['how many', 'count', 'number of ways', 'arrangements', 'permutation', 'combination']:
        if kw in q: scores[ProblemType.COMBINATORICS] += 2
    for kw in ['triangle', 'circle', 'square', 'area', 'perimeter', 'angle', 'point', 'line']:
        if kw in q: scores[ProblemType.GEOMETRY] += 2
    for kw in ['equation', 'polynomial', 'root', 'solve', 'sequence', 'series']:
        if kw in q: scores[ProblemType.ALGEBRA] += 2
    
    sorted_types = sorted(scores.items(), key=lambda x: -x[1])
    primary = sorted_types[0][0] if sorted_types[0][1] >= 2 else ProblemType.MIXED
    
    mod_target = None
    mod_match = re.search(r'(?:mod|modulo)\s*(\d+)', q)
    if mod_match:
        mod_target = int(mod_match.group(1))
    
    return ProblemProfile(primary, mod_target, min(1.0, sorted_types[0][1]/10.0))

print("Classifier ready")

In [ ]:
# Cell 7: Code Execution Stdlib
STDLIB = '''
import sys; sys.setrecursionlimit(20000)
import math, numpy as np
from itertools import *
from collections import *
from functools import lru_cache, reduce
from fractions import Fraction
try:
    import sympy
    from sympy import *
    from sympy.ntheory import factorint, divisors, totient, isprime
except: pass

def is_prime(n):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0: return False
    return True

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def C(n, k):
    if k < 0 or k > n: return 0
    return math.factorial(n) // (math.factorial(k) * math.factorial(n - k))
'''

SNOOP = '''
_vars = dict(globals())
for _v in ['answer', 'result', 'ans', 'res', 'final', 'output', 'solution', 'total', 'count']:
    if _v in _vars and _vars[_v] is not None:
        try: print(int(_vars[_v])); break
        except: pass
'''

print("Stdlib ready")

In [ ]:
# Cell 8: Answer Processing
def normalize_answer(raw: Any) -> Optional[int]:
    if raw is None: return None
    try:
        s = str(raw).replace(',', '').replace(' ', '').strip()
        s = re.sub(r'[^0-9.eE+-]', '', s)
        if not s: return None
        val = float(s)
        if val != val or abs(val) == float('inf'): return None
        result = int(round(val))
        if result < 0: result = abs(result)
        return result % (ANSWER_MAX + 1)
    except: return None

def extract_python_code(text: str) -> Optional[str]:
    for pattern in [r'```python\s*\n(.*?)```', r'```py\s*\n(.*?)```', r'```\s*\n(.*?)```']:
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if matches: return matches[-1].strip()
    return None

def extract_boxed(text: str) -> Optional[int]:
    for pattern in [r'\\boxed\{([^}]+)\}', r'boxed\{([^}]+)\}', r'[Aa]nswer\s*(?:is|=|:)\s*([0-9]+)']:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches: return normalize_answer(matches[-1])
    return None

def execute_code(code: str, retries: int = 2) -> Tuple[Optional[int], str]:
    if not code: return None, ""
    has_print = 'print(' in code
    full = STDLIB + "\n" + code + ("" if has_print else "\n" + SNOOP)
    timeout = min(30, 10 + len(re.findall(r'\bfor\b|\bwhile\b', code)) * 5)
    
    for attempt in range(retries + 1):
        try:
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
                f.write(full)
                f.flush()
                result = subprocess.run(['python', f.name], capture_output=True, text=True, timeout=timeout)
                os.unlink(f.name)
                
                if result.returncode == 0 and result.stdout.strip():
                    numbers = re.findall(r'-?\d+\.?\d*', result.stdout)
                    if numbers: return normalize_answer(numbers[-1]), code
        except subprocess.TimeoutExpired: pass
        except: pass
        break
    return None, code

print("Answer processing ready")

In [ ]:
# Cell 9: Basin Selection (CIC-inspired)
def relative_distance(a: float, b: float) -> float:
    if a == b: return 0.0
    if a == 0 or b == 0: return 1.0
    return abs(a - b) / max(abs(a), abs(b))

def select_answer(candidates: List[int], mod_target: Optional[int] = None) -> int:
    """CIC Basin Clustering - 88% error reduction."""
    if not candidates: return FALLBACK
    if len(candidates) == 1: return candidates[0]
    
    if mod_target:
        candidates = [c % mod_target for c in candidates]
    
    # Cluster by proximity (5% threshold)
    basins = defaultdict(list)
    for c in candidates:
        assigned = False
        for bval in list(basins.keys()):
            if relative_distance(c, bval) < 0.05:
                basins[bval].append(c)
                assigned = True
                break
        if not assigned:
            basins[c].append(c)
    
    # Score basins
    basin_scores = [(len(members), int(statistics.median(members))) for members in basins.values()]
    basin_scores.sort(key=lambda x: -x[0])
    final = basin_scores[0][1]
    
    # Avoid trivial 0/1
    if final in (0, 1) and len(candidates) > 3:
        non_trivial = [c for c in candidates if c not in (0, 1)]
        if len(non_trivial) >= 2:
            return select_answer(non_trivial, mod_target)
    
    return max(0, min(final, ANSWER_MAX))

print("Basin selection ready")

In [ ]:
# Cell 10: Prompts - PROMETHEUS-style
SYSTEM_PROMPTS = {
    'pot_algorithmic': (
        "You are an expert mathematical problem solver. Write self-contained Python code. "
        "Use int() not float. Use pow(a,b,m) for modular exponentiation. "
        "Enclose code in ```python ... ```. Print ONLY the integer answer."
    ),
    'pot_sympy': (
        "You are a symbolic computation expert. Use sympy for EXACT arithmetic. "
        "Use Rational(a,b) NOT a/b. Never use floats. Verify with simplify(). "
        "Enclose code in ```python ... ```. Print ONE integer."
    ),
    'pot_bruteforce': (
        "You are a computational enumerator. Systematically enumerate all cases. "
        "Budget: 10^7 iterations max. Use itertools and memoization. "
        "Enclose code in ```python ... ```. Print ONE integer."
    ),
    'cot': (
        "You are a mathematical olympiad expert. Think step-by-step. "
        "Check your work. Final answer in \\boxed{answer}."
    ),
}

TYPE_HINTS = {
    ProblemType.NUMBER_THEORY: "\nHINT: NUMBER THEORY - consider modular arithmetic, divisibility, primes, CRT, Fermat.",
    ProblemType.COMBINATORICS: "\nHINT: COMBINATORICS - counting, binomials, inclusion-exclusion, generating functions.",
    ProblemType.GEOMETRY: "\nHINT: GEOMETRY - coordinates, distance formula, shoelace, similar triangles.",
    ProblemType.ALGEBRA: "\nHINT: ALGEBRA - polynomials, Vieta's, sequences, systems of equations.",
    ProblemType.MIXED: "",
}

def create_prompt(question: str, strategy: str, profile: ProblemProfile) -> List[Dict]:
    system = SYSTEM_PROMPTS.get(strategy, SYSTEM_PROMPTS['pot_algorithmic'])
    type_hint = TYPE_HINTS.get(profile.primary_type, "")
    mod_hint = f"\nIMPORTANT: Final answer must be mod {profile.modulo_target}." if profile.modulo_target else ""
    user = f"Problem: {question}{type_hint}{mod_hint}\n\nSolve this. Print ONLY the integer answer."
    return [{"role": "system", "content": system}, {"role": "user", "content": user}]

print("Prompts ready")

In [ ]:
# Cell 11: Batch Generation
POT_STOP = ["```output", "```\nOutput", "# Output", "\n\n\n"]
COT_STOP = ["</think>", "\n\nQuestion:"]

def batch_generate(messages: List[List[Dict]], mode: str = 'pot', temp: float = 0.6) -> List[str]:
    if not llm or not tokenizer:
        return []
    prompts = [tokenizer.apply_chat_template(m, tokenize=False, add_generation_prompt=True) for m in messages]
    params = SamplingParams(
        temperature=temp, top_p=0.9, min_p=0.05, max_tokens=4096,
        skip_special_tokens=True, stop=POT_STOP if mode == 'pot' else COT_STOP
    )
    outputs = llm.generate(prompts, sampling_params=params)
    return [o.outputs[0].text for o in outputs]

def process_outputs(outputs: List[str]) -> List[int]:
    candidates = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {}
        for i, out in enumerate(outputs):
            code = extract_python_code(out)
            if code:
                futures[executor.submit(execute_code, code)] = i
            else:
                ans = extract_boxed(out)
                if ans is not None:
                    candidates.append(ans)
        
        for future in as_completed(futures):
            try:
                ans, _ = future.result()
                if ans is not None:
                    candidates.append(ans)
            except: pass
    return candidates

print("Generation ready")

In [ ]:
# Cell 12: Panic Mode
def panic_guess(question: str, profile: ProblemProfile) -> int:
    q = question.lower()
    numbers = [int(x) for x in re.findall(r'\b\d+\b', question) if 0 < int(x) < ANSWER_MAX]
    
    if profile.modulo_target:
        if numbers: return numbers[0] % profile.modulo_target
        return 0
    if 'how many' in q:
        return min(numbers) if numbers else 1
    if 'sum' in q:
        small = [n for n in numbers if n < 1000]
        return sum(small) % (ANSWER_MAX + 1) if small else 0
    return numbers[0] % (ANSWER_MAX + 1) if numbers else FALLBACK

print("Panic mode ready")

In [ ]:
# Cell 13: Main Solver - ADAPTIVE SINGLE-PASS + TEMPERATURE ANNEALING
# Start HOT each problem (diverse exploration)
# Cool DOWN as difficulty detected (convergent refinement)

PROBLEM_COUNT = 0
SOLVED_COUNT = 0
TOTAL_PROBLEMS_EST = 50

def get_adaptive_config(time_remaining: float, problem_idx: int) -> dict:
    """Adaptive sampling based on time budget."""
    time_frac = time_remaining / TOTAL_BUDGET
    
    if time_frac > 0.7:
        return {"pot_samples": 6, "cot_samples": 4, "max_tokens": 4096}
    elif time_frac > 0.4:
        return {"pot_samples": 4, "cot_samples": 2, "max_tokens": 3072}
    elif time_frac > 0.15:
        return {"pot_samples": 3, "cot_samples": 1, "max_tokens": 2048}
    else:
        return {"pot_samples": 2, "cot_samples": 0, "max_tokens": 1536}

def estimate_difficulty(question: str, profile: ProblemProfile) -> float:
    """Estimate problem difficulty 0-1 for temperature scaling."""
    q = question.lower()
    difficulty = 0.3  # Base
    
    # Length indicator
    if len(question) > 500: difficulty += 0.15
    if len(question) > 1000: difficulty += 0.15
    
    # Complexity keywords
    hard_keywords = ['prove', 'minimize', 'maximize', 'optimal', 'least', 'greatest', 
                     'all possible', 'exactly', 'unique', 'exists']
    for kw in hard_keywords:
        if kw in q: difficulty += 0.08
    
    # Multiple constraints
    constraint_words = ['such that', 'where', 'given that', 'if and only if', 'for all']
    for cw in constraint_words:
        if cw in q: difficulty += 0.05
    
    # Mixed types are harder
    if profile.primary_type == ProblemType.MIXED: difficulty += 0.1
    
    return min(1.0, difficulty)

def anneal_temperature(base_temp: float, phase: int, difficulty: float) -> float:
    """Temperature annealing: start hot, cool as phases progress and difficulty increases."""
    # Phase cooling: 0.9, 0.75, 0.6 multipliers
    phase_mult = [1.0, 0.85, 0.7][min(phase, 2)]
    # Difficulty cooling: hard problems need more focused search
    diff_mult = 1.0 - (difficulty * 0.3)
    return max(0.3, base_temp * phase_mult * diff_mult)

def solve_question(question: str) -> int:
    global PROBLEM_COUNT, SOLVED_COUNT
    PROBLEM_COUNT += 1
    
    time_remaining = CUTOFF_TIME - time.time()
    profile = classify_problem(question)
    config = get_adaptive_config(time_remaining, PROBLEM_COUNT)
    difficulty = estimate_difficulty(question, profile)
    
    # START HOT - diverse exploration
    base_temp = 0.85
    
    print(f"\n[Q{PROBLEM_COUNT}] {question[:70]}...")
    print(f"  Type: {profile.primary_type.value} | Diff: {difficulty:.2f} | Mod: {profile.modulo_target}")
    print(f"  Time: {time_remaining/60:.1f}m | Samples: PoT×{config['pot_samples']} CoT×{config['cot_samples']}")
    
    if time_remaining < PANIC_TIME:
        print("  ⚠️ PANIC")
        return panic_guess(question, profile)
    
    candidates = []
    
    # === PHASE 0: HOT START - maximum diversity ===
    temp_p0 = anneal_temperature(base_temp, 0, difficulty)
    print(f"  Phase 0: HOT PoT×{config['pot_samples']} (T={temp_p0:.2f})")
    
    strategies = ['pot_algorithmic', 'pot_sympy', 'pot_bruteforce']
    messages = [create_prompt(question, strategies[i % 3], profile) 
                for i in range(config['pot_samples'])]
    
    outputs = batch_generate(messages, 'pot', temp=temp_p0)
    phase0_ans = process_outputs(outputs)
    candidates.extend(phase0_ans)
    print(f"    → {len(phase0_ans)}: {phase0_ans[:6]}")
    
    # EARLY EXIT on strong consensus
    if len(candidates) >= 3:
        counts = Counter(candidates)
        best, count = counts.most_common(1)[0]
        if count >= 3:
            print(f"  ✓ CONSENSUS: {best} ({count}/{len(candidates)})")
            SOLVED_COUNT += 1
            gc.collect(); torch.cuda.empty_cache()
            return int(best)
    
    # === PHASE 1: WARM - focused CoT (if enabled) ===
    if config['cot_samples'] > 0 and time_remaining > 120:
        temp_p1 = anneal_temperature(base_temp, 1, difficulty)
        print(f"  Phase 1: WARM CoT×{config['cot_samples']} (T={temp_p1:.2f})")
        
        cot_msgs = [[
            {"role": "system", "content": SYSTEM_PROMPTS['cot']},
            {"role": "user", "content": f"{question}\n\nSolve carefully. Answer in \\boxed{{}}"}
        ] for _ in range(config['cot_samples'])]
        
        cot_outputs = batch_generate(cot_msgs, 'cot', temp=temp_p1)
        for out in cot_outputs:
            ans = extract_boxed(out)
            if ans is not None:
                candidates.append(ans)
        print(f"    → Now {len(candidates)} total")
    
    # === PHASE 2: COOL - refinement if no consensus yet ===
    if len(candidates) >= 2 and time_remaining > 180:
        counts = Counter(candidates)
        best, count = counts.most_common(1)[0]
        if count < len(candidates) * 0.5:  # No majority yet
            temp_p2 = anneal_temperature(base_temp, 2, difficulty)
            print(f"  Phase 2: COOL refine×2 (T={temp_p2:.2f})")
            
            # Targeted refinement on top candidates
            refine_msgs = [create_prompt(question, 'pot_algorithmic', profile) for _ in range(2)]
            refine_outputs = batch_generate(refine_msgs, 'pot', temp=temp_p2)
            refine_ans = process_outputs(refine_outputs)
            candidates.extend(refine_ans)
            print(f"    → +{len(refine_ans)}: {refine_ans}")
    
    # === FINAL SELECTION ===
    if not candidates:
        print("  ⚠️ No answers")
        return panic_guess(question, profile)
    
    counts = Counter(candidates)
    best, count = counts.most_common(1)[0]
    conf = count / len(candidates)
    
    if conf >= 0.5:
        final = int(best)
        print(f"  ✓ FINAL: {final} ({count}/{len(candidates)} = {conf:.0%})")
    else:
        final = select_answer(candidates, profile.modulo_target)
        print(f"  → BASIN: {final} (weak consensus)")
    
    SOLVED_COUNT += 1
    gc.collect(); torch.cuda.empty_cache()
    return final

print("Annealing solver ready")

In [ ]:
# Cell 14: Kaggle API Interface - EXACT FORMAT FOR SUBMISSION
def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: Optional[pl.DataFrame] = None) -> pl.DataFrame:
    """
    Kaggle AIMO3 API - EXACT FORMAT REQUIRED FOR SUBMISSION.
    
    Args:
        id_: polars DataFrame with problem ID
        question: polars DataFrame with problem text
        answer: Optional ground truth (for local testing)
    
    Returns:
        pl.DataFrame with columns 'id' and 'answer'
    """
    id_val = id_.item(0)
    question_str = question.item(0)
    
    time_left = (CUTOFF_TIME - time.time()) / 60
    print(f"\n{'='*60}")
    print(f"ID: {id_val} | Time: {time_left:.1f}m | Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")
    
    try:
        prediction = solve_question(question_str)
        prediction = max(ANSWER_MIN, min(ANSWER_MAX, int(prediction)))
    except Exception as e:
        print(f"ERROR: {e}")
        prediction = FALLBACK
    
    print(f"ANSWER: {prediction}")
    print(f"{'='*60}")
    
    return pl.DataFrame({'id': id_val, 'answer': prediction})

print("Kaggle API ready")

In [ ]:
# Cell 15: START COMPETITION
print("="*70)
print("AIMO3 Elle Submission - 11 Dec 2025")
print("="*70)
print(f"Model: {LLM_MODEL_PATH}")
print(f"Budget: {TOTAL_BUDGET//60} minutes")
print(f"Sample mode: {IS_SAMPLE}")
print("="*70)

# Create inference server with our predict function
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("COMPETITION MODE - serving...")
    inference_server.serve()
else:
    print("LOCAL TEST MODE")
    inference_server.run_local_gateway((f'{COMP_PATH}/test.csv',))
    print(f"\n{'='*60}")
    print(f"FINAL: Solved {SOLVED_COUNT}/{PROBLEM_COUNT}")
    print(f"{'='*60}")